# Heart Disease

In [68]:
import torch
from torch.utils.data import Dataset,  TensorDataset, DataLoader, random_split
from torch import nn
import numpy as np

## Crear Dataset

In [69]:
heart_csv = np.genfromtxt("./heart.csv",
              delimiter=",",
              dtype=[('data',"float16",(13,)),('target','int')],
              skip_header=1,
            )
data = heart_csv['data']
target = heart_csv['target']
data = np.array(data, dtype=np.float32).copy()
target = np.array(target, dtype=np.int64).copy()

print(data[0])
data = (data - np.min(data, axis=0, keepdims=True)) / np.max(data, axis=0, keepdims=True)
print(data[0])

[ 52.   1.   0. 125. 212.   0.   1. 168.   0.   1.   2.   2.   3.]
[0.2987013  1.         0.         0.155      0.15248227 0.
 0.5        0.48019803 0.         0.16131064 1.         0.5
 1.        ]


## To Tensor Dataset

In [70]:
data_tensor = torch.tensor(data, dtype=torch.float32) # X
target_tensor = torch.tensor(target, dtype=torch.long) # Y

class HeartDiseaseDataset(Dataset):
    def __init__(self, x, y, transform):
        self.x = x
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.transform(read_image(self.x[idx])), self.y[idx]



def divide_by_255(x):
    return x/255
    


    

dataset = TensorDataset(data_tensor,target_tensor)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
training_data, test_data = random_split(dataset, [train_size, test_size])


## Dataloader

In [71]:
bs = 10

train_dataloader = DataLoader(training_data, batch_size=bs, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=bs, shuffle=True)

In [72]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(13, 100),
            nn.ReLU(),
            nn.Linear(100, 20),
            nn.ReLU(),
            nn.Linear(20, 2)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
   
model = NeuralNetwork().to(device)
print(model)  

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=13, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=2, bias=True)
  )
)


In [73]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Limpiar gradientes antes de calcularlos
        optimizer.zero_grad()

        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            print(f"loss: {loss.item():>7f}")



def test_loop(dataloader, model, loss_fn, device):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
           
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \nAccuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [74]:
def entrenarModelo(learning_rate, epochs):
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train_loop(train_dataloader, model, loss_fn, optimizer, device)
        test_loop(test_dataloader, model, loss_fn, device)

    print("\nModelo entrenado!!")

In [81]:
entrenarModelo(0.001, 100)

Epoch 1
-------------------------------
loss: 0.351798
Test Error: 
Accuracy: 98.5%, Avg loss: 0.034482 

Epoch 2
-------------------------------
loss: 0.001946
Test Error: 
Accuracy: 99.0%, Avg loss: 0.031469 

Epoch 3
-------------------------------
loss: 0.001179
Test Error: 
Accuracy: 99.0%, Avg loss: 0.029144 

Epoch 4
-------------------------------
loss: 0.001331
Test Error: 
Accuracy: 99.0%, Avg loss: 0.026478 

Epoch 5
-------------------------------
loss: 0.005496
Test Error: 
Accuracy: 99.0%, Avg loss: 0.025235 

Epoch 6
-------------------------------
loss: 0.056874
Test Error: 
Accuracy: 99.5%, Avg loss: 0.025116 

Epoch 7
-------------------------------
loss: 0.041459
Test Error: 
Accuracy: 99.5%, Avg loss: 0.020752 

Epoch 8
-------------------------------
loss: 0.056043
Test Error: 
Accuracy: 99.0%, Avg loss: 0.021593 

Epoch 9
-------------------------------
loss: 0.002378
Test Error: 
Accuracy: 99.0%, Avg loss: 0.017839 

Epoch 10
-------------------------------
loss:

In [76]:
def saveTrainedModel():
    torch.save(model.state_dict(), 'heartDisease.pth')

In [77]:
def chargeTrainedModel():
    model.load_state_dict(torch.load('heartDisease.pth'))
    model.eval()

In [78]:
def prediccionInput(input_data):
    print(input_data)
    input_tensor = (input_data - np.min(input_data, axis=0, keepdims=True)) / np.max(input_data, axis=0, keepdims=True)
    print(input_tensor)
    input_tensor = torch.tensor(input_data, dtype=torch.float32).unsqueeze(0).to(device)
    print(f'INPUT: {input_tensor}')
    with torch.no_grad():
        output = model(input_tensor)
        probabilities = torch.softmax(output, dim=1)
        predicted_class = probabilities.argmax(1).item()

    print(f"Clase predicha: {predicted_class}")

In [79]:
def main(learning_rate, epochs):
    entrenarModelo(learning_rate, epochs)
    saveTrainedModel()
    print("Introduce tu edad:")
    edad = int(input())
    print(edad)

    print("Introduce tu sexo (0 para mujer, 1 para hombre):")
    sexo = int(input())
    print(sexo)

    print("Introduce el tipo de dolor torácico (0-3):")
    tipo_dolor_toracico = int(input())
    print(tipo_dolor_toracico)

    print("Introduce tu presión arterial en reposo (en mm Hg):")
    presion_arterial = int(input())
    print(presion_arterial)

    print("Introduce tu nivel de colesterol (en mg/dl):")
    colesterol = int(input())
    print(colesterol)

    print("Introduce tu nivel de glucosa en ayunas (1 si > 120 mg/dl, 0 si no):")
    glucosa_ayunas = int(input())
    print(glucosa_ayunas)

    print("Introduce el resultado del electrocardiograma en reposo (0-2):")
    ecg_reposo = int(input())
    print(ecg_reposo)

    print("Introduce tu frecuencia cardíaca máxima alcanzada:")
    frecuencia_cardiaca = int(input())
    print(frecuencia_cardiaca)

    print("¿Tuviste angina inducida por ejercicio? (1 para sí, 0 para no):")
    angina_ejercicio = int(input())
    print(angina_ejercicio)

    print("Introduce el valor de depresión del ST después del ejercicio (oldpeak):")
    oldpeak = float(input())
    print(oldpeak)

    print("Introduce la pendiente del segmento ST (0-2):")
    pendiente_ST = int(input())
    print(pendiente_ST)

    print("Introduce el número de vasos principales coloreados por fluoroscopía (0-3):")
    vasos_coloreados = int(input())
    print(vasos_coloreados)

    print("Introduce el tipo de talasemia (1 = normal, 2 = defecto fijo, 3 = defecto reversible):")
    talasemia = int(input())
    print(talasemia)

    input_data = [edad,sexo,tipo_dolor_toracico,presion_arterial, colesterol,glucosa_ayunas,ecg_reposo,frecuencia_cardiaca,
                  angina_ejercicio,oldpeak,pendiente_ST,vasos_coloreados,talasemia]

    
    chargeTrainedModel()
   
    prediccionInput(input_data)


learning_rate = 0.01
epochs = 30
main(learning_rate, epochs)


Epoch 1
-------------------------------
loss: 0.302276
Test Error: 
Accuracy: 86.8%, Avg loss: 0.277586 

Epoch 2
-------------------------------
loss: 0.169777
Test Error: 
Accuracy: 87.8%, Avg loss: 0.287338 

Epoch 3
-------------------------------
loss: 0.187199
Test Error: 
Accuracy: 88.8%, Avg loss: 0.239349 

Epoch 4
-------------------------------
loss: 0.149206
Test Error: 
Accuracy: 93.2%, Avg loss: 0.207248 

Epoch 5
-------------------------------
loss: 0.418153
Test Error: 
Accuracy: 85.4%, Avg loss: 0.303363 

Epoch 6
-------------------------------
loss: 0.291133
Test Error: 
Accuracy: 89.3%, Avg loss: 0.220956 

Epoch 7
-------------------------------
loss: 0.196422
Test Error: 
Accuracy: 92.2%, Avg loss: 0.207413 

Epoch 8
-------------------------------
loss: 0.052465
Test Error: 
Accuracy: 87.8%, Avg loss: 0.258170 

Epoch 9
-------------------------------
loss: 0.082486
Test Error: 
Accuracy: 91.2%, Avg loss: 0.181067 

Epoch 10
-------------------------------
loss:

 65


65
Introduce tu sexo (0 para mujer, 1 para hombre):


 1


1
Introduce el tipo de dolor torácico (0-3):


...

 2


2
Introduce el tipo de talasemia (1 = normal, 2 = defecto fijo, 3 = defecto reversible):


 3


3
[65, 1, 3, 150, 240, 1, 2, 120, 1, 2.5, 2, 2, 3]
[0.26666667 0.         0.00833333 0.62083333 0.99583333 0.
 0.00416667 0.49583333 0.         0.00625    0.00416667 0.00416667
 0.00833333]
INPUT: tensor([[ 65.0000,   1.0000,   3.0000, 150.0000, 240.0000,   1.0000,   2.0000,
         120.0000,   1.0000,   2.5000,   2.0000,   2.0000,   3.0000]],
       device='cuda:0')
Clase predicha: 1


/tmp/ipykernel_23529/928787273.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('heartDisease.pth'))
